In [14]:
import dr_util.file_utils as fu
from omegaconf import DictConfig, OmegaConf
from hydra import initialize, compose

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
with initialize(config_path="../configs/", version_base=None):
    cfg = compose(config_name="paper_data.yaml")
print(OmegaConf.to_yaml(OmegaConf.to_container(cfg, resolve=True)))

data_dir: /Users/daniellerothermel/drotherm/data/
raw_pdf_dir: /Users/daniellerothermel/drotherm/data/raw_pdfs/
parsed_pdf_dir: /Users/daniellerothermel/drotherm/data/parsed_pdfs/
metadata_dir: /Users/daniellerothermel/drotherm/data/parsed_pdfs/
author_data_dir: /Users/daniellerothermel/drotherm/data/author_data/
author_summaries_dir: /Users/daniellerothermel/drotherm/data/author_data/summaries/
arxiv_dir: /Users/daniellerothermel/drotherm/data/arxiv/
author_info_file: /Users/daniellerothermel/drotherm/data/author_data/manual_profiles.json
arxiv_metadata_file: /Users/daniellerothermel/drotherm/data/arxiv/arxiv-metadata-oai-2024-10-14.json
prof_pattern: (?P<professor_name>[\w_]+)
file_type_pattern: (?P<file_type>\w+)
version_pattern: v(?P<version>\d+)
author_summary_file_pattern: (?P<professor_name>[\w_]+)\.(?P<file_type>\w+)\.v(?P<version>\d+)



In [26]:
print(fu.fu_help())


:: Help for dr_util.file_utils ::
 --------------------------------- 

 For pathlib helpers: fu.pathlib_help()

 Main Functions
  - load_file(path, force_suffix=None, mmm=None, *, verbose=True)
  - dump_file(data, path, force_suffix=None, verbose=Tue)
  - load_files(path_list)

 Supported Endings: json, jsonl, pkl, txt, npy, yaml

 --------------------------------- 



In [29]:
md = fu.load_file(cfg.arxiv_metadata_file, force_suffix='jsonl')

In [30]:
len(md)

15000

In [32]:
md[0].keys()

dict_keys(['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'abstract', 'report-no', 'categories', 'versions', 'versions_dates'])